In [1]:
from langchain.tools import Tool
from langchain_community.llms import LlamaCpp
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Dict, Any, List, Tuple, TypedDict
from langgraph.graph import Graph, StateGraph
from typing import List, Dict, Tuple, Any
import ast
import re

mistral = LlamaCpp(
    model_path="C:/Users/DaysPC/Langgraph/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.1,
    max_tokens=2000,
    n_ctx=32768,
    top_p=0.95,
    n_batch=64,
    verbose=False
)


def search_web(query: str) -> str:
    """Simulate web search."""
    return f"Found results for: {query}"

def calculate(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {str(e)}"

def write_file(content: str) -> str:
    """Simulate writing content to a file."""
    return f"Successfully wrote: {content}"

In [2]:
class PlanAgent:
    def __init__(self, model: LlamaCpp):
        self.model = model

    def create_plan(self, query: str) -> List[Dict[str, Any]]:
        """Break down the query into 3 structured tasks."""

        prompt = f"""[INST] Break this task into exactly 3 simple steps:
Task: {query}

Create 3 separate tasks:
1. First find/search for the required information
2. Then perform the calculation or action on that information
3. Finally, analyze or explain the trend or historical change related to the query

Format as a Python list with exactly 3 dictionaries:
[
    {{"task_id": 1, "description": "Search for specific information", "status": "pending"}},
    {{"task_id": 2, "description": "Perform calculation with the found information", "status": "pending"}},
    {{"task_id": 3, "description": "Analyze how it has changed over time", "status": "pending"}}
]

Return only the Python list.[/INST]</s>"""

        try:
            response = self.model.invoke(prompt)
            response = response.strip()
            tasks = ast.literal_eval(response)
            
            if not isinstance(tasks, list) or len(tasks) != 3:
                raise ValueError("Model did not return exactly 3 tasks.")
            
            cleaned_tasks = []
            for i, task in enumerate(tasks, 1):
                if not isinstance(task, dict) or "description" not in task:
                    raise ValueError(f"Invalid task format at index {i}")
                cleaned_tasks.append({
                    "task_id": i,
                    "description": task["description"],
                    "status": "pending"
                })

            return cleaned_tasks

        except Exception as e:

            return [
                {
                    "task_id": 1,
                    "description": f"Search for information: {query}",
                    "status": "pending"
                },
                {
                    "task_id": 2,
                    "description": "Process the found information",
                    "status": "pending"
                },
                {
                    "task_id": 3,
                    "description": "Analyze how it changed historically",
                    "status": "pending"
                }
            ]

In [3]:
import re
from typing import Any, Dict, List

class ToolAgent:
    def __init__(self, tools: List, model):
        self.model = model
        self.calculator = next(t for t in tools if t.name == "calculator")
        self.current_population = None
        self.population_history = {}
        self.location = None

    def _extract_location(self, text: str) -> str:
        prompt = f"""[INST] Extract only the location name from the following user query:
{text}
Return only the location name, nothing else.[/INST]"""
        result = self.model.invoke(prompt).strip()
        if not result or "unable" in result.lower() or len(result.split()) > 5:
            return "New York City"
        return result

    def _get_current_population(self, location: str) -> int:
        prompt = f"What is the current population of {location} in 2021?"
        response = self.model.invoke(prompt)
        response_text = response if isinstance(response, str) else str(response)

        match = re.search(r"(\d{1,3}(?:,\d{3})+)", response_text)
        if match:
            try:
                return int(match.group(1).replace(",", ""))
            except ValueError:
                pass

        match_million = re.search(r"([\d.]+)\s*(million|billion)", response_text, re.IGNORECASE)
        if match_million:
            number = float(match_million.group(1))
            scale = match_million.group(2).lower()
            multiplier = 1_000_000 if scale == "million" else 1_000_000_000
            return int(number * multiplier)

        raise ValueError(f"Could not extract population from: {response_text}")

    def _get_population_history(self, location: str) -> Dict[str, int]:
        prompt = f"""[INST] Give population estimates of {location} in 1990, 2000, and 2010.
Only include the year and number in each sentence. Use millions if necessary.[/INST]"""
        response = self.model.invoke(prompt)
        response_text = getattr(response, "content", str(response))

        matches = re.findall(r'(\d{4}).*?([\d.]+)\s*million', response_text, re.IGNORECASE)
        if not matches:
            raise ValueError("Could not parse historical population.")

        return {year: int(float(pop_mil) * 1_000_000) for year, pop_mil in matches}

    def execute_task(self, task: Dict[str, Any]) -> Dict[str, Any]:
        try:
            task_id = task.get("task_id")

            if task_id == 1:
                desc = task.get("description", "")
                location = self._extract_location(desc)
                pop = self._get_current_population(location)

                if not (100_000 <= pop <= 2_000_000_000):
                    raise ValueError("Population value out of expected range")

                self.current_population = pop
                self.location = location

                return {
                    **task,
                    "status": "completed",
                    "result": f"Population of {location}: {pop:,}",
                    "tool_used": "language_model"
                }

            elif task_id == 2:
                if self.current_population is None:
                    raise ValueError("Missing population data from Task 1")
                calc = f"{self.current_population} * 0.15"
                result = self.calculator.run(calc)
                return {
                    **task,
                    "status": "completed",
                    "result": f"15% of {self.current_population:,} = {float(result):,.2f}",
                    "tool_used": "calculator"
                }

            elif task_id == 3:
                if not self.location:
                    raise ValueError("Location not found from previous task")

                history = self._get_population_history(self.location)
                self.population_history = history

                diffs = []
                for year, past_pop in history.items():
                    growth = self.current_population - past_pop
                    percent = (growth / past_pop) * 100
                    diffs.append(f"{year} → Now: Growth = {growth:,} ({percent:.1f}%)")

                summary = "\n".join(diffs)
                return {
                    **task,
                    "status": "completed",
                    "result": f"Population growth for {self.location}:\n{summary}",
                    "tool_used": "model_history_analysis"
                }

            else:
                raise ValueError(f"Unknown task_id: {task_id}")

        except Exception as e:
            return {
                **task,
                "status": "failed",
                "error": str(e),
                "result": f"Task failed due to error: {str(e)}"
            }

    def reflect_on_result(self, task: Dict[str, Any]) -> str:
        if task["status"] == "completed":
            if task["task_id"] == 1:
                return "Retrieved current population data successfully."
            elif task["task_id"] == 2:
                return "Performed calculation successfully."
            elif task["task_id"] == 3:
                return "Historical analysis completed."
        return f"Task {task['task_id']} failed: {task.get('error', 'Unknown error')}"


In [4]:
class WorkflowState(TypedDict):
    query: str
    tasks: List[Dict[str, Any]]
    current_task_id: int
    completed: bool
    steps: int
    last_node: str


def create_workflow(tools: List[Tool], model: LlamaCpp) -> Graph:
    """Create an advanced automated workflow for sequential task execution."""

    plan_agent = PlanAgent(model)
    tool_agent = ToolAgent(tools, model)

    def plan(state: WorkflowState) -> WorkflowState:
        """Create a plan from query if not already present."""
        if not state["tasks"]:
            state["tasks"] = plan_agent.create_plan(state["query"])
            state["steps"] = 0
            state["last_node"] = "plan"
            state["current_task_id"] = 1
        return state

    def execute(state: WorkflowState) -> WorkflowState:
        """Execute current task using the appropriate tool."""
        state["steps"] += 1
        state["last_node"] = "execute"

        current_task = next(
            (t for t in state["tasks"] if t["task_id"] == state["current_task_id"]),
            None
        )

        if current_task and current_task["status"] == "pending":
            result = tool_agent.execute_task(current_task)

            state["tasks"] = [
                result if t["task_id"] == current_task["task_id"] else t
                for t in state["tasks"]
            ]

            if state["current_task_id"] < len(state["tasks"]):
                state["current_task_id"] += 1
            else:
                state["completed"] = True

        return state

    def should_continue(state: WorkflowState) -> Tuple[bool, str]:
        """Determine whether to continue workflow or stop."""
        if state["completed"]:
            return False, "end"

        if state["steps"] >= 10:
            state["completed"] = True
            return False, "end"

        if not state["tasks"]:
            return True, "plan"

        current_task = next(
            (t for t in state["tasks"] if t["task_id"] == state["current_task_id"]),
            None
        )

        if current_task and current_task["status"] == "pending":
            return True, "execute"

        if state["current_task_id"] < len(state["tasks"]):
            return True, "execute"

        state["completed"] = True
        return False, "end"

    workflow = StateGraph(WorkflowState)

    workflow.add_node("plan", plan)
    workflow.add_node("execute", execute)

    workflow.set_entry_point("plan")

    workflow.add_conditional_edges("plan", should_continue)
    workflow.add_conditional_edges("execute", should_continue)

    return workflow.compile()


def run_workflow(graph: Graph, query: str) -> Dict[str, Any]:
    """Run a compiled workflow with a given query."""
    initial_state: WorkflowState = {
        "query": query,
        "tasks": [],
        "current_task_id": 1,
        "completed": False,
        "steps": 0,
        "last_node": ""
    }
    return graph.invoke(initial_state)


In [5]:
tools = [
    Tool(
        name="web_search",
        func=search_web,
        description="Search the web for information."
    ),
    Tool(
        name="calculator",
        func=calculate,
        description="Calculate mathematical expressions."
    ),
    Tool(
        name="file_writer",
        func=write_file,
        description="Write content to a file."
    )
]

try:
    workflow = create_workflow(tools, mistral)

    query = "Find the population of New York City and calculate 15% of it, How has it changed over the years"

    result = run_workflow(workflow, query)

    print("\nWorkflow completed!")
    print("\nTasks and Results:")
    for task in result["tasks"]:
        print(f"\nTask {task['task_id']}:")
        print(f"Description: {task['description']}")
        print(f"Status: {task['status']}")
        if "result" in task:
            print(f"Result: {task['result']}")

except Exception as e:
    print(f"\nError during workflow execution: {str(e)}")


Workflow completed!

Tasks and Results:

Task 1:
Description: Search for specific information
Status: completed
Result: Population of New York City: 8,336,817

Task 2:
Description: Perform calculation with the found information
Status: completed
Result: 15% of 8,336,817 = 1,250,522.55

Task 3:
Description: Analyze how it has changed over time
Status: completed
Result: Population growth for New York City:
1990 → Now: Growth = 836,817 (11.2%)
2000 → Now: Growth = 236,817 (2.9%)
2010 → Now: Growth = -63,183 (-0.8%)
